In [1]:
import numpy as np
import pandas as pd

# Cargo la base que ya limpiamos y exportamos en el modelo 1
df = pd.read_csv('df_cleaned.csv')

# Me aseguro de que no haya precios nulos o no positivos
df = df[df['price'] > 0].copy()

# Target en log(precio) para estabilizar la varianza
df['log_price'] = np.log(df['price'])

# Tomo solo columnas numéricas (incluye dummies 0/1)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Quito la columna de precio original y el log_price del set de features
numeric_cols.remove('price')
numeric_cols.remove('log_price')

# Matrices finales para el modelo
X = df[numeric_cols].values
y = df['log_price'].values

print("Shape X:", X.shape)
print("Shape y:", y.shape)
print("Número de features:", len(numeric_cols))


Shape X: (20827, 98)
Shape y: (20827,)
Número de features: 98


In [2]:
from sklearn.model_selection import train_test_split

# Parto en train y test (dejo 20% para test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape, y_test.shape)

# Importo TensorFlow (igual que en el taller)
import setuptools.dist  # parche para Python 3.12
import tensorflow as tf

# Normalización sobre los datos de entrenamiento
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(X_train)

n_features = X_train.shape[1]
print("Features que ve la red:", n_features)


Train: (16661, 98) (16661,)
Test : (4166, 98) (4166,)
Features que ve la red: 98
